In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data

from torch.nn import *
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.nn import Sequential
import os
from torchvision.datasets import DatasetFolder
import sys

In [2]:
import torch.nn as nn
import pickle
from torchvision.datasets import ImageFolder

In [3]:
#my files
from MyResNet import *

In [4]:
basepath = '.'
train_path = basepath + '/train_data'
train_medium_path = train_path + '/medium'

val_path = basepath + '/validation_classification'
val_medium_path = val_path + '/medium'

test_path = basepath + '/test_classification'
test_medium_path = test_path + '/medium'

In [5]:
train_transform = transforms.Compose([
        transforms.CenterCrop(28),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
val_transform = transforms.Compose([
        transforms.CenterCrop(28),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
#validation using normal transform
val_imagefolder = ImageFolder(val_medium_path, transform = val_transform)
#use more transforms for train dataset to get robust 
train_imagefolder = ImageFolder(train_medium_path, transform = train_transform)

In [8]:
validation_dataloader = DataLoader(val_imagefolder,
                                    batch_size = 32,
                                    drop_last = False,
                                    shuffle = False)

In [9]:
train_dataloader = DataLoader(train_imagefolder,
                                    #when use 32 bits, 128 batch size works fine
                                    batch_size = 32,
                                    drop_last = True,
                                    shuffle = True,
                                    num_workers = 0)

In [10]:
def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

In [11]:
from PIL import Image

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [12]:
class TestImageFolder(DatasetFolder):
    
    def __init__(self, root, extensions=None, transform=None):
        super(DatasetFolder, self).__init__(root, transform=transform)
        
        self.root = root
        self.loader = default_loader
        self.extensions = extensions
        self.samples = list(os.walk(root))[0][2]
    

    def __getitem__(self, index):
        
        file_name = self.samples[index]
        sample = self.loader(os.path.join(self.root, file_name))
        if self.transform is not None:
            sample = self.transform(sample)

        return sample


    def __len__(self):
        return len(self.samples)

In [13]:
def validation_acc(model, dataloader):
    
    model.eval().cuda()
    
    with torch.no_grad(): 
        y_pred = []
        y_true = []
        for index, (features, labels) in enumerate(dataloader):
            features = features.cuda()
            #labels = labels.cuda()
            batch_size = features.shape[0]

            outputs = model.forward(features).view(batch_size, -1)
            preds = torch.max(outputs.data, 1)[1]
            Y_p += list(preds.data.cpu().numpy())
            Y_t += list(labels.numpy())
        
    return sum(np.array(Y_p) == np.array(Y_t))/len(Y_t)

In [14]:
def validation_acc(model, dataloader):
    
    model.eval().cuda()
    
    with torch.no_grad(): 
        y_pred = []
        y_true = []
        for index, (features, labels) in enumerate(dataloader):
            features = features.cuda()
            #labels = labels.cuda()
            batch_size = features.shape[0]

            outputs = model(features).view(batch_size, -1)
            preds = torch.max(outputs.data, 1)[1]
            Y_p += list(preds.data.cpu().numpy())
            Y_t += list(labels.numpy())
        
    return sum(np.array(Y_p) == np.array(Y_t))/len(Y_t)

In [15]:
def ensemble_validation_acc(models, dataloader):
    
    with torch.no_grad(): 
        for model in models:
            model.eval().cuda()
        
        Y_p = []
        Y_t = []
        for index, (features, labels) in enumerate(dataloader):
            
            features = features.cuda()
            #labels = labels.cuda()
            batch_size = features.shape[0]

            outputs = torch.zeros((batch_size, 2300))
            for model in models:
                outputs += model(features).view(batch_size, -1).cpu()
            
            preds = torch.max(outputs.data, 1)[1]
            Y_p += list(preds.data.cpu().numpy())
            Y_t += list(labels.numpy())
            
    return sum(np.array(y_true) == np.array(y_pred))/len(y_true)

## Train models

In [ ]:
if __name__ == '__main__':
    model = MyResNet(BasicBlock, [1, 2, 3, 4])
    #downsample less in the first few layers
    # since I used mean value of several resnet, I trained five models.
    '''
    model = MyResNet(BasicBlock, [2, 2, 3, 2])
    model = MyResNet(BasicBlock, [2, 3, 3, 2])
    model = MyResNet(BasicBlock, [1, 3, 3, 2])
    model = MyResNet(BasicBlock, [1, 2, 4, 1])
    '''
    n_epoches = 15
    criterion = CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),
                                lr = 0.05,
                                weight_decay = 5e-6,
                                momentum = 0.9)
    
    train_dataloader = train_dataloader
    validation_dataloader = validation_dataloader
    check_iter = 500
    #start epoch
    for epoch in range(n_epoches):
        print("Epoch {0} starts.".format(epoch+1))
        model = model.train().cuda()
        temp_avg_loss = 0
        last_avg_loss = 1e10
        batch_size = train_dataloader.batch_size
        
        for index, (features, targets) in enumerate(train_dataloader):
            
            optimizer.zero_grad()
            
            #to cuda
            features = features.cuda()
            targets = targets.cuda()
            
            model.cuda()
            outputs = model(features).view(batch_size, -1)
            
            loss = criterion(outputs, targets.long())
            loss.backward()
            #sum loss
            temp_avg_loss += loss.item()
            optimizer.step()
            
            if (index+1)%check_iter == 0:
                temp_avg_loss /= check_iter
                print(temp_avg_loss)
                if temp_avg_loss >= last_avg_loss:
                    # if the average loss between two check points increase, decay the learning rate
                    for param in optimizer.param_groups:
                        param['lr'] /= 2
                    print("Decay learning rate to {0}.".format(optimizer.param_groups[0]['lr']))
        print("Epoch {0} ends.".format(epoch+1))
        #validate the result
        print(validation_acc(model, validation_dataloader))
        
    #change file name when store different models
    torch.save(model, "SingleModel1.pt")

## Ensemble Results

### 1.Classification

In [ ]:
model_list = []
#file_names = ["SingleModel1.pt", "SingleModel2.pt", "SingleModel3.pt", "SingleModel4.pt", "SingleModel5.pt"]
file_names = ["SingleModel1.pt"]
              
for file_name in file_names:
    model = torch.load(file_name)
    model_list.append(model)

ensemble_validation_accuracy = ensemble_validation_acc(model_list, validation_dataloader)
print("Ensemble validation accuracy is", ensemble_validation_accuracy)

# This generates the final results

In [ ]:
def make_ensemble_classification_submission(model_list, idx2class):
    
    # TestImageFolder is defined in previous section
    batch_size = 64
    test_datafolder = TestImageFolder(test_medium_path, transform = val_transform)
    test_dataloader = DataLoader(test_datafolder, 
                             batch_size = batch_size,
                             drop_last = False,
                             shuffle = False)
    
    sub_index = test_dataloader.dataset.samples
    sub_preds = []
    
    for model in model_list:
        model.eval().cuda()
        
    for index, (features) in enumerate(test_dataloader):
        
        features = features.cuda()
        
        outputs = torch.zeros((features.shape[0], 2300))
        for model in model_list:
            outputs += model.forward(features).view(features.shape[0], -1).cpu()
            
        preds = torch.max(outputs.data, 1)[1]
        sub_preds += list(preds.data.cpu().numpy())
    
    sub_preds = [idx2class[i] for i in sub_preds]
    submission_df = pd.DataFrame({'Category': sub_preds}, index = sub_index)
    submission_df.to_csv('submission.csv', index_label = 'Id')

In [ ]:
#map the prediction to real class
idx2class = {idx:cls for cls, idx in val_imagefolder.class_to_idx.items()}

In [ ]:
make_ensemble_classification_submission(model_list, idx2class)

### 2.Verification

In [ ]:
def parse_data(img_path, transforms, base_path):
    img = Image.open(base_path + '/' + img_path)
    img = val_transform(img)
    return img

In [ ]:
class VerificationImageDataset(Dataset):
    def __init__(self, veri_file, transforms = val_transform, test_mode = False):
        
        self.veri_file = veri_file
        self.transforms = transforms
        self.img1 = [i.split()[0] for i in open(veri_file, 'r').readlines()]
        self.img2 = [i.split()[1] for i in open(veri_file, 'r').readlines()]
        self.test_mode = test_mode
        if test_mode == True:
            self.labels = [-1]*len(open(veri_file, 'r').readlines())
        else:
            self.labels = [int(i.split()[2]) for i in open(veri_file, 'r').readlines()]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if self.test_mode == False:
            return parse_data(self.img1[index], self.transforms, val_veri_path), parse_data(self.img2[index], self.transforms, val_veri_path), torch.Tensor([self.labels[index]])
        else:
            return parse_data(self.img1[index], self.transforms, test_veri_path), parse_data(self.img2[index], self.transforms, test_veri_path), torch.Tensor([self.labels[index]])

In [ ]:
verification_dataset = VerificationImageDataset("validation_trials_verification.txt")

In [ ]:
veri_dataloader = DataLoader(verification_dataset,
                            batch_size = 128,
                            drop_last = False,
                            shuffle = False)

In [ ]:
from torch.nn.functional import cosine_similarity

def make_veri_prediction(model):
    #using cosine similarity, no fine-tuning
    model.eval()
    td = VerificationImageDataset("test_trials_verification_student.txt", test_mode=True)
    test_dataloader = DataLoader(td,
                            batch_size = 100,
                            drop_last = False,
                            shuffle = False)
    simi = []
    
    for i, (img1, img2, labels) in enumerate(test_dataloader):
        output_img1 = model(img1.cuda())
        output_img2 = model(img2.cuda())

        simi.append(cosine_similarity(output_img1, output_img2).detach().cpu().numpy())
        del output_img1
        del output_img2
    
    total_simi = np.concatenate(simi)
    
    output = pd.DataFrame({'trial':open("test_trials_verification_student.txt").readlines(), 'score':total_simi})
    output.to_csv("verification_output.csv")

In [ ]:
model = torch.load("SingleModel1.pt")

In [ ]:
base_path = ''
val_veri_path = base_path + 'validation_verification'
val_veri_txt_path = base_path + 'validation_trials_verification.txt'

test_veri_path = base_path + 'test_verification/test_verification'
test_veri_txt_path = base_path + 'test_trials_verification_student.txt'

In [ ]:
make_veri_prediction(model)